In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter  # 导入 tensorboard 所需使用的包
import numpy as np
import torch.nn.functional as F


# 定义LeNet-5模型
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 加载CIFAR10数据集
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

data_path = '../data-unversioned/p1ch7/'

trainset = torchvision.datasets.CIFAR10(root=data_path, train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=True, num_workers=2)
train_len=len(trainset)

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
testset = torchvision.datasets.CIFAR10(root=data_path, train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)
logdir = './log'


writer = SummaryWriter(log_dir=logdir)
# 初始化LeNet-5模型和优化器
net = LeNet5()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# 训练模型
for epoch in range(1):
    running_loss = 0.0
    total_accurate=0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        pred = outputs.squeeze()
        a = np.array(pred.data)

        # 对模型的输出进行 softmax 计算
        probs = F.softmax(outputs, dim=1)
        # 将真实标签转换为 one-hot 编码
        one_hot_labels = torch.zeros(outputs.size()).scatter_(1, labels.unsqueeze(1), 1)
        # 计算交叉熵损失
        loss_s = -torch.sum(one_hot_labels * torch.log(probs), dim=1)
        loss_m = torch.mean(loss_s)

        loss = criterion(outputs, labels)
        loss_zero = loss_s - loss
        # print(loss_zero)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        accurate=((pred>0.5)==labels).sum().item()
        total_accurate+=accurate
        # 在 epoch 循环中
        avg_acc=total_accurate/train_len
        writer.add_scalar("train_loss", loss, epoch)  # 在每次计算完 loss 后添加
        # writer.add_scalar("val_acc", val_avg_acc, epoch)
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

# 测试模型
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
# 循环结束后
writer.close()
#  tensorboard --logdir ./p1ch8/log 运行相应web应用，读取相应log文件

[1,   200] loss: 2.307
[1,   400] loss: 2.289
[1,   600] loss: 2.165
[1,   800] loss: 2.193
[1,  1000] loss: 2.130
[1,  1200] loss: 2.042
[1,  1400] loss: 2.030
[1,  1600] loss: 2.011
[1,  1800] loss: 2.003
[1,  2000] loss: 2.079
[1,  2200] loss: 2.082
[1,  2400] loss: 2.019
[1,  2600] loss: 1.947
[1,  2800] loss: 1.976
[1,  3000] loss: 2.035
[1,  3200] loss: 1.983


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x107a62160>
Traceback (most recent call last):
  File "/Users/xuzhao/Desktop/ml/dlwpt-code/venv/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1466, in __del__
    self._shutdown_workers()
  File "/Users/xuzhao/Desktop/ml/dlwpt-code/venv/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1430, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/connection.py", line 936, in wait
    ready = selector.select(timeout)
  File "/Library/Frameworks/Python.framework/

KeyboardInterrupt: 